In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
% matplotlib inline
from sklearn.preprocessing import StandardScaler
import glob

In [50]:
from importlib import reload
from sln import utils; reload(utils)
from sln import train; reload(train)

<module 'sln.train' from 'D:\\side\\CarND-Vehicle-Detection\\sln\\train.py'>

In [26]:
vehicles = utils.load_array('vehicles/')
non_vehicles = utils.load_array('non_vehicles/')

In [51]:
def generate_feats(imgs):
    feats = []
    for img in imgs:
        feat = train.generate_feat(img, 'LUV')
        feats.append(feat)
    return np.array(feats)

vehicles_feats = generate_feats(vehicles)
non_vehicles_feats = generate_feats(non_vehicles)

In [52]:
y = np.hstack((np.ones(len(vehicles_feats)), np.zeros(len(non_vehicles_feats))))
x = np.vstack((vehicles_feats, non_vehicles_feats))
x_scaler = StandardScaler().fit(x)
scaled_x = x_scaler.transform(x)

In [53]:
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train.train_test_split(
    scaled_x, y, test_size=0.2, random_state=rand_state)

In [54]:
svc = train.LinearSVC()
svc.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [55]:
print('Test Accuracy of SVC = ', svc.score(X_test, y_test))

Test Accuracy of SVC =  0.990990990991


In [56]:
import pickle
with open('svc.p', 'wb') as f:
    pickle.dump({'svc': svc, 'scaler': x_scaler}, f)